<a href="https://colab.research.google.com/github/kyaoNK/bert_nli/blob/main/CREATE_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!sudo apt -y update
# !sudo apt -y -q install libeigen3-dev libprotobuf-dev protobuf-c-compiler

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://archive.ubuntu.com/ubuntu b

In [2]:
!pip install -qU pytorch-lightning lightning-bolts 
!pip install -qU torch
!pip install -qU torchtext
!pip install -qU torchvision
!pip install -qU torchaudio
!pip install -q   datasets
!pip install -q   accelerate
!pip install -q   ipadic
!pip install -q   unidic-lite
!pip install -qU transformers
!pip install -q   sentencepiece
# !pip install -qU sentence_transformers
# !pip install -q   pyknp
# !pip install -qU torch==1.7.1
# !pip install -qU torchtext==0.8.0
# !pip install -qU torchvision==0.8.2
# !pip install -qU torchaudio==0.7.2
# !pip install -q   transformers==4.4.2
# !pip install -q   pytorch_ligtning==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.0/330.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 887.4/887.4 MB 120.3 MB/s eta 0:00:01tcmalloc: large alloc 1109270528 bytes == 0x389ba000 @  0x7f57088ea615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')

### Juman++

In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/nlp-resource/JUMANPP/jumanpp-2.0.0-rc3
# !mkdir build
# %cd build
# !rm -f CMakeCache.txt
# !cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
# !make
# !sudo make install
# !cd /content

In [ ]:
# from pyknp import Juman
# jumanpp = Juman()   # default is JUMAN++: Juman(jumanpp=True). if you use JUMAN, use Juman(jumanpp=False)
# result = jumanpp.analysis("下鴨神社の参道は暗かった。")
# for mrph in result.mrph_list(): # 各形態素にアクセス
#     print("見出し:%s, 読み:%s, 原形:%s, 品詞:%s, 品詞細分類:%s, 活用型:%s, 活用形:%s, 意味情報:%s, 代表表記:%s" \
#             % (mrph.midasi, mrph.yomi, mrph.genkei, mrph.hinsi, mrph.bunrui, mrph.katuyou1, mrph.katuyou2, mrph.imis, mrph.repname))

# Datasets

In [4]:
import csv

def tsv2csv (tsv_filepath) :
    with open(tsv_filepath, 'r') as tsv_f:
        tsv = csv.reader(tsv_f, delimiter = '\t')
        csv_filepath = tsv_filepath[:-4] + '.csv'
        with open(csv_filepath, 'w', newline='\n') as csv_f:
            writer = csv.writer(csv_f, delimiter=",")
            for row in tsv:
                writer.writerow(row)
            return csv_filepath

In [19]:
#@title JGLUE RTE
import pandas as pd

JGLUE_DIR = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/JGLUE/datasets/jnli-v1.1/'
JGLUE_FILENAME = { 'train' : 'train-v1.1.json' , 'valid' : 'valid-v1.1.json' }
JGLUE_COLS = [ 'yjcaptions_id', 'sentence1' , 'sentence2' , 'label' ]
COLUMNS = [ 'id', 'sentenceA' , 'sentenceB' , 'label_rte' ]

jglue_rte_train_df = pd.read_json( JGLUE_DIR + JGLUE_FILENAME['train'] , lines=True , encoding='utf-8' )
jglue_rte_train_df = jglue_rte_train_df.loc[ : , JGLUE_COLS ]
jglue_rte_train_df.columns = COLUMNS
jglue_rte_valid_df = pd.read_json( JGLUE_DIR + JGLUE_FILENAME['valid'] , lines=True , encoding='utf-8' )
jglue_rte_valid_df = jglue_rte_valid_df.loc[ : , JGLUE_COLS ]
jglue_rte_valid_df.columns = COLUMNS

In [ ]:
jglue_rte_train_df

In [21]:
#@title JGLUE STS
import pandas as pd

JGLUE_DIR = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/JGLUE/datasets/jsts-v1.1/'
JGLUE_FILENAME = { 'train' : 'train-v1.1.json' , 'valid' : 'valid-v1.1.json' }
JGLUE_COLS = [ 'yjcaptions_id' , 'sentence1' , 'sentence2' , 'label' ]
COLUMNS = [ 'id', 'sentenceA' , 'sentenceB' , 'label_sts' ]

jglue_sts_train_df = pd.read_json( JGLUE_DIR + JGLUE_FILENAME['train'] , lines=True , encoding='utf-8' )
jglue_sts_train_df = jglue_sts_train_df.loc[ : , JGLUE_COLS ]
jglue_sts_train_df.columns = COLUMNS
jglue_sts_valid_df = pd.read_json( JGLUE_DIR + JGLUE_FILENAME['valid'] , lines=True , encoding='utf-8' )
jglue_sts_valid_df = jglue_sts_valid_df.loc[ : , JGLUE_COLS ]
jglue_sts_valid_df.columns = COLUMNS

In [ ]:
jglue_sts_train_df

In [44]:
#@title JGLUE MERGE
import pandas as pd

jglue_df = pd.merge(jglue_rte_train_df, jglue_sts_train_df[['id', 'label_sts']], on='id')
jglue_df

,id,sentenceA,sentenceB,label_rte,label_sts
0,100124-104404-104405,二人の男性がジャンボジェット機を見ています。,2人の男性が、白い飛行機を眺めています。,neutral,3.8
1,100142-104431-104432,男性が子供を抱き上げて立っています。,坊主頭の男性が子供を抱いて立っています。,neutral,4.0
2,100282-104568-104569,皿の上にサンドイッチが盛られています。,白い皿の上に切られたサンドイッチと野菜料理が載っています。,neutral,2.6
3,10041-11051-11053,スキー場のゲレンデで滑っている人がいます。,広大な雪上をスキーで滑っていきます。,neutral,3.8
4,100417-104710-104712,クマが木に登って枝にまたがっています。,一頭の熊が松と思われる木の上にいます。,neutral,3.8
...,...,...,...,...,...
7221,99718-104051-104052,数人の人がいる中、海辺で野球をやっている人がいます。,砂浜で、野球の練習をしている少年です。,neutral,2.2
7222,99785-104096-104098,顔にペイントした少年二人が、ベッドに寝そべっています。,子供が顔にペイントして寝転んでいます。,entailment,3.8
7223,99952-104226-104228,広い室内にたくさんの人々が椅子に座っています。,広い室内にたくさんの人たちが椅子に座っています。,entailment,4.4
7224,99952-104227-104228,広い室内でたくさんの人たちが椅子に座っています。,広い室内にたくさんの人たちが椅子に座っています。,entailment,4.8


In [49]:
#@title NEW LABEL
label_df = jglue_df[[ 'label_rte' , 'label_sts' ]]
# label_df['label_detail'] = None
def function(x):
    # print(x['label_rte'], x['label_sts'])
    if x['label_rte'] == 'entailment' :
        if x['label_sts'] >= 4.5 :
            new_label = '正解'
        elif x['label_sts'] >= 3.5 :
            new_label = '異なる修飾'
        elif x['label_sts'] >= 2.5 
            new_label = '欠如'
        else :
            new_label = '不十分'
    if x['label_rte'] == 'contradiction' :
        if x['label_sts'] >= 3.5 :
            new_label = '異なる修飾'
        else :
            new_label = '矛盾'
    if x['label_rte'] == 'neutral' :
        if x['label_sts'] >= 2.5 :
            new_label = '無関係'
        else :
            new_label = 'ドメイン外'
            
    return pd.Series( [ x['label_rte'] , x['label_sts'] , new_label ] )

label_df = label_df.apply(function, axis=1)
label_df

,0,1,2
0,neutral,3.8,無関係
1,neutral,4.0,無関係
2,neutral,2.6,無関係
3,neutral,3.8,無関係
4,neutral,3.8,無関係
...,...,...,...
7221,neutral,2.2,ドメイン外
7222,entailment,3.8,異なる修飾
7223,entailment,4.4,異なる修飾
7224,entailment,4.8,正解


In [ ]:
#@title JSICK
import pandas as pd

JSICK_DIRPATH = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/JSICK/jsick/'
JSICK_FILENAME = { 'train' : 'train.tsv' , 'valid' : 'test.tsv' }
JSICK_COLS = [ 'sentence_A_Ja' , 'sentence_B_Ja' , 'entailment_label_Ja' ]
COLUMNS = [ 'sentenceA' , 'sentenceB' , 'label' ]

jsick_train_filepath = tsv2csv(JSICK_DIRPATH + JSICK_FILENAME['train'])
jsick_valid_filepath = tsv2csv(JSICK_DIRPATH + JSICK_FILENAME['valid'])

jsick_train_df = pd.read_csv( jsick_train_filepath , encoding='utf-8' , header=0 )
jsick_train_df = jsick_train_df.loc[ : , JSICK_COLS ]
jsick_train_df.columns = COLUMNS
jsick_valid_df = pd.read_csv( jsick_valid_filepath , encoding='utf-8' , header=0 )
jsick_valid_df = jsick_valid_df.loc[ : , JSICK_COLS ]
jsick_valid_df.columns = COLUMNS

jsick_train_df.index = len(jglue_train_df.index) + jsick_train_df.index
jsick_valid_df.index = len(jglue_valid_df.index) + jsick_valid_df.index

# jsick_train_df